In [1]:
#いろいろインポート
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR

pd.set_option('display.width',400)#勝手に改行コードを入れられるのを防ぐ
place='suminoe'

In [2]:
#モデルスコアデータの読み込み
#score_df = pd.read_csv('../csv/model_score_date_{}.csv'.format(place))#dateアリのモデル
score_df = pd.read_csv('../csv/{}_model_score_normal.csv'.format(place))#dateなしのモデル
score_df=score_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
display(score_df)

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数
0,1.0,4.0,51.0,354000.0,406900.0,331.0,4069.0,86.999263,8.134677,331.0
1,1.0,4.0,52.0,354000.0,406900.0,331.0,4069.0,86.999263,8.134677,331.0
2,1.0,4.0,53.0,354000.0,406900.0,331.0,4069.0,86.999263,8.134677,331.0
3,1.0,4.0,54.0,354000.0,406900.0,331.0,4069.0,86.999263,8.134677,331.0
4,1.0,4.0,55.0,354000.0,406900.0,331.0,4069.0,86.999263,8.134677,331.0
...,...,...,...,...,...,...,...,...,...,...
13855,28.0,8.0,145.0,18580.0,24500.0,19.0,245.0,75.836735,2.857143,7.0
13856,28.0,8.0,146.0,8850.0,24200.0,19.0,242.0,36.570248,2.066116,5.0
13857,28.0,8.0,147.0,15490.0,22800.0,19.0,228.0,67.938596,2.192982,5.0
13858,28.0,8.0,148.0,16540.0,23800.0,19.0,238.0,69.495798,2.521008,6.0


#### モデル性能評価用のデータフレーム作成(result_comごとの閾値を持つ)


In [3]:
#学習データの読み込み
result_df = pd.read_csv('../csv/train_{}_date.csv'.format(place))
result_df=result_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除


df=result_df.copy()

#学習、テストデータ切り分け(dateを使って学習データを切り分ける。)

df['date']=pd.to_datetime(df['date'])#文字列なのでdateを日付型に変換
df['year']=df['date'].dt.year
valid_df = df[(df['year']==2019) | ((df['year']==2020) )]#2019,2020のデータを検証用データに。
train_df =  df[(df['year']!=2019) & ((df['year']!=2020) )]#そのほかを学習データに
#学習データを切り分けたらdateとyearはいらないから削除する
valid_df=valid_df.drop(['year'],axis=1)
train_df=train_df.drop(['year'],axis=1)
valid_df=valid_df.drop(['date'],axis=1)
train_df=train_df.drop(['date'],axis=1)

train_money=pd.Series(train_df['money'])
valid_money=pd.Series(valid_df['money'])

#出現数の分布
result_com_s=valid_df['result_com'].value_counts()
result_com_s=result_com_s.sort_index()
gain_mean=valid_df.groupby('result_com')['money'].mean()
gain_mean=gain_mean.sort_index()

gain_median=valid_df.groupby('result_com')['money'].median()
gain_median=gain_median.sort_index()
result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                            'result_com_num':result_com_s.values,
                            'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                            'gain_mean':gain_mean.values,
                            'gain_median':gain_median.values,})
display(result_com_df)

,result_com,result_com_num,result_com_per,gain_mean,gain_median
0,1,331,8.134677,1069.486405,820.0
1,2,250,6.144016,1457.920000,965.0
2,3,154,3.784714,2160.064935,1270.0
3,4,130,3.194888,2351.000000,1580.0
4,5,278,6.832145,1386.294964,1080.0
...,...,...,...,...,...
114,116,2,0.049152,19980.000000,19980.0
115,117,3,0.073728,14260.000000,9200.0
116,118,4,0.098304,14757.500000,10905.0
117,119,4,0.098304,14870.000000,14645.0


### モデル性能評価機能

In [4]:
#基準をクリアしたモデルを格納するdf
depths_arr=[4,5,6,7,8]
model_para_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
for target_com in result_com_df['result_com'].values:
    target_com_df=score_df[score_df['target_com']==target_com]
    #閾値の作成===============================================================
    gain_th=110#利益率の閾値
    result_s=result_com_df[result_com_df['result_com']==target_com]
    buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
    num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
    
    diff_gain_th=108#利益率の閾値
    diff_buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#前後モデルの買ったうちの的中率の閾値
    diff_num_tp_th=result_s['result_com_num'].values[0]*0.13#前後モデルのあたった回数の閾値(出現回数の20%が的中)
    
    #==========================================================
    for target_depth in depths_arr:
        target_com_depth_df=target_com_df[target_com_df['depth']==target_depth]
        
        len_df=len(target_com_depth_df)
        df=target_com_depth_df.copy()
        df['number_i']=np.arange(0,len_df,1)
        
        for _, row in df.iterrows():
            if ((row['number_i']==0) or (row['number_i']==1) or (row['number_i']==len_df-2) or (row['number_i']==len_df-1)):
                pass
            else:
                #前後のモデルのスコア===================================
                diff_m2_row=df[df['number_i']==(row['number_i']-2)]
                diff_m1_row=df[df['number_i']==(row['number_i']-1)]
                diff_p1_row=df[df['number_i']==(row['number_i']+1)]
                diff_p2_row=df[df['number_i']==(row['number_i']+2)]
                diff_models=[diff_m2_row,diff_m1_row,diff_p1_row,diff_p2_row]
                #===================================================
                if ((row['利益率']>gain_th) and (row['購買的中率']>buy_accuracy_th) and (row['的中数']>=num_tp_th)):#初めに真ん中のモデルの性能評価
                    #前後モデルの評価
                    flag_arr=[0,0,0,0]
                    for index in range(len(flag_arr)):

                        diff_model=diff_models[index].iloc[0,:]
                        if ((diff_model['利益率']>diff_gain_th) and (diff_model['購買的中率']>diff_buy_accuracy_th) and (diff_model['的中数']>=diff_num_tp_th)):#前後モデルの性能評価
                            flag_arr[index]=1
                        else:
                            pass
                    if sum(flag_arr)==4:
                        model_para_df=model_para_df.append(row)
                        
                else:
                    pass
        

In [5]:
model_para_df

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i
2038,5.0,4.0,109.0,86170.0,71300.0,278.0,713.0,120.855540,11.079944,79.0,58.0
2039,5.0,4.0,110.0,65480.0,56000.0,278.0,560.0,116.928571,10.892857,61.0,59.0
2138,5.0,5.0,110.0,88480.0,80000.0,278.0,800.0,110.600000,10.500000,84.0,59.0
2139,5.0,5.0,111.0,85900.0,70300.0,278.0,703.0,122.190612,11.379801,80.0,60.0
2140,5.0,5.0,112.0,82840.0,60300.0,278.0,603.0,137.379768,12.271973,74.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...
13195,27.0,7.0,79.0,213850.0,182900.0,29.0,1829.0,116.921815,1.148168,21.0,28.0
13196,27.0,7.0,80.0,211300.0,177100.0,29.0,1771.0,119.311124,1.129305,20.0,29.0
13197,27.0,7.0,81.0,211300.0,177100.0,29.0,1771.0,119.311124,1.129305,20.0,30.0
13198,27.0,7.0,82.0,213850.0,168200.0,29.0,1682.0,127.140309,1.248514,21.0,31.0


In [6]:
#model_para_df.to_csv('../csv/{}_good_score_model_date.csv'.format(place), encoding='utf_8_sig')
model_para_df.to_csv('../csv/{}_good_score_model_normal.csv'.format(place), encoding='utf_8_sig')

In [7]:
diff_model

target_com       27.000000
depth             8.000000
target_per      146.000000
総収益           12260.000000
投資金額          26200.000000
出現数              29.000000
購買予測数           262.000000
利益率              46.793893
購買的中率             1.908397
的中数               5.000000
number_i         95.000000
Name: 13361, dtype: float64

In [8]:
len_df

0

In [9]:
type(diff_model)

pandas.core.series.Series